In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'src')))
import numpy as np
import pandas as pd
import segyio
import matplotlib.pyplot as plt
from pathlib import Path
from matplotlib.pyplot import figure
import matplotlib.colors as mcolors
from so3d.utils.segy_reader import *
from so3d.utils.array_signal_plotter import plot_receiver_signals, freq_plot, freq_plot_tcut, plot_seismic,plot_shot_source_signal,plot_signal_mf_comparison
from so3d.utils.data_utils import get_shot_data, get_line_data, get_receivers_data,select_line_data_portion,match_filter
from so3d.utils.color_utils import redblue, redblue_cmap
import scipy.io

In [2]:
# read data
data_file = '../artifacts/raw/R1809SA8S299.sgy'
data, delta_t, textheader_output = read_segy(data_file)

No of traces = 15315, No of samples per trace = 3200


In [4]:
# Constants
traces_per_shot = 5105
receivers_per_line = 638
num_lines = 8

########################################### Functions get_shot_data ########################################
def get_shot_data(shot_no, data):
    """
    Retrieve data for a specific shot, excluding the first receiver.
    Args:
        shot_no (int): The shot number (1-indexed).
        data (numpy.ndarray): The entire dataset.
    Returns:
        numpy.ndarray: Data for the specified shot.
    """
    start_idx = (shot_no - 1) * traces_per_shot + 1  # Skip the first receiver
    end_idx = shot_no * traces_per_shot
    return data[start_idx:end_idx]

(15315, 3200)

In [10]:
import os
import numpy as np
from obspy import read

# Constants
traces_per_shot = 5105
receivers_per_line = 638
num_lines = 8

def process_sgy_files(input_folder, output_folder):
    """
    Read SEG-Y files from the input folder, extract line data, and save it into the output folder.

    Args:
        input_folder (str): Path to the folder containing .sgy files.
        output_folder (str): Path to the folder where the line data will be saved.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate over all .sgy files in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith('.sgy'):
            # Load the data from the SEG-Y file
            data_file = os.path.join(input_folder, filename)
            data, delta_t, textheader_output = read_segy(data_file, textheader='no')
            
            num_shots = data.shape[0] // traces_per_shot  # Total number of shots

            # Extract and save data for each line within each shot
            for shot_no in range(1, num_shots + 1):
                for line_no in range(1, num_lines + 1):
                    line_data = get_line_data(shot_no, line_no, data)
                    output_filename = f"{os.path.splitext(filename)[0]}_shot_{shot_no}_line_{line_no}.npy"
                    output_path = os.path.join(output_folder, output_filename)

                    # Save the line data to a .npy file
                    np.save(output_path, line_data)

                    print(f"Saved: {output_path}")

# # Example usage
# input_folder = 'path/to/input/folder'  # Specify your input folder here
# output_folder = 'path/to/output/folder'  # Specify your output folder here

# process_sgy_files(input_folder, output_folder)



In [11]:
# Example usage
input_folder = '../artifacts/raw'  # Specify your input folder here
output_folder = '../artifacts/segmented'  # Specify your output folder here

process_sgy_files(input_folder, output_folder)

No of traces = 15315, No of samples per trace = 3200
Saved: ../artifacts/segmented\R1809SA8S299_shot_1_line_1.npy
Saved: ../artifacts/segmented\R1809SA8S299_shot_1_line_2.npy
Saved: ../artifacts/segmented\R1809SA8S299_shot_1_line_3.npy
Saved: ../artifacts/segmented\R1809SA8S299_shot_1_line_4.npy
Saved: ../artifacts/segmented\R1809SA8S299_shot_1_line_5.npy
Saved: ../artifacts/segmented\R1809SA8S299_shot_1_line_6.npy
Saved: ../artifacts/segmented\R1809SA8S299_shot_1_line_7.npy
Saved: ../artifacts/segmented\R1809SA8S299_shot_1_line_8.npy
Saved: ../artifacts/segmented\R1809SA8S299_shot_2_line_1.npy
Saved: ../artifacts/segmented\R1809SA8S299_shot_2_line_2.npy
Saved: ../artifacts/segmented\R1809SA8S299_shot_2_line_3.npy
Saved: ../artifacts/segmented\R1809SA8S299_shot_2_line_4.npy
Saved: ../artifacts/segmented\R1809SA8S299_shot_2_line_5.npy
Saved: ../artifacts/segmented\R1809SA8S299_shot_2_line_6.npy
Saved: ../artifacts/segmented\R1809SA8S299_shot_2_line_7.npy
Saved: ../artifacts/segmented\R1